In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np

import torch
from torchvision import datasets, transforms

In [ ]:
MEANS = [0.5 , 0.5, 0.5]
DEVIATIONS = [0.5, 0.5 , 0.5]
transform = transforms.Compose([transforms.Resize((255,255)),
                                transforms.Resize((224,224)),
                                transforms.RandomHorizontalFlip(p=0.5),
                                transforms.ToTensor(),
                                transforms.Normalize(MEANS,DEVIATIONS)
                               ])


In [ ]:
train_dataset = datasets.ImageFolder('./drive/My Drive/data/train', transform=transform)
vali_dataset = datasets.ImageFolder('./drive/My Drive/data/validation', transform=transform)

In [ ]:
print('train :',len(train_dataset))
print('validation :',len(vali_dataset))

train : 14327
validation : 1620


In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
vali_dataloader = torch.utils.data.DataLoader(vali_dataset, batch_size=64, shuffle=True)

In [ ]:
import torchvision.models as models

model_transfer = models.resnext101_32x8d(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth" to /root/.cache/torch/hub/checkpoints/resnext101_32x8d-8ba56ff5.pth


In [ ]:
for param in model_transfer.parameters():
    param.requires_grad = False

In [ ]:
from collections import OrderedDict
fc = nn.Sequential(OrderedDict([
                          ('(0):Dropout' , nn.Dropout()),
                          ('(1):Linear', nn.Linear(2048, 1000)),
                          ('(2):Batchnorm', nn.BatchNorm1d(1000)), 
                          ('(3):relu', nn.ReLU()),
                          ('(4):Dropout' , nn.Dropout()),
                          ('(5):Linear', nn.Linear(1000, 100)),
                          ('(6):Batchnorm', nn.BatchNorm1d(100)),
                          ('(7):relu', nn.ReLU()),
                          ('(8):Linear', nn.Linear(100,10)),
    
                          ('(9):Batchnorm', nn.BatchNorm1d(10)),
                          ('(10):Linear', nn.Linear(10,5)),
                          ('(11):relu', nn.ReLU()),
    
                          ('(12):output', nn.LogSoftmax(dim=1))
                          ]))

model_transfer.fc = fc

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model_transfer.parameters() , lr =1e-3)

In [ ]:
Epoch = 6

In [ ]:
vali_loss_min = np.inf
vali_loss_second = np.inf

train_losses,valid_losses=[],[]
for epoch in range(Epoch):
    print(f'{epoch+1}st training start')
    
    train_accuracy =0
    vali_accuracy = 0
    train_loss ,vali_loss =0,0
    
    
    model_transfer.train()
    for x, y in train_dataloader:
        
        output= model_transfer(x)
        
        optimizer.zero_grad()
        
        loss = criterion(output, y)
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()*x.size(0)
        
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == y.view(*top_class.shape)
        train_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
    model_transfer.eval()
    with torch.no_grad():
        for x, y in vali_dataloader:

            output= model_transfer(x)
            loss=criterion( output, y)

            vali_loss += loss.item()*x.size(0)

            ps = torch.exp(output)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == y.view(*top_class.shape)
            vali_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        

            
        train_loss = train_loss/ len(train_dataloader.sampler)
        vali_loss = vali_loss/ len(vali_dataloader.sampler)

        train_accuracy=(train_accuracy/len(train_dataloader))*100
        vali_accuracy=(vali_accuracy/len(vali_dataloader))*100
        
        
        train_losses.append(train_loss)
        valid_losses.append(vali_loss)
        
        
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss, vali_loss))
        print('          \tTraining Accuracy: {:.2f} \tValidation Accuracy: {:.2f}%'.format( train_accuracy, vali_accuracy))

        
        
        if vali_loss < vali_loss_second:
            if vali_loss < vali_loss_min:
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format( vali_loss_min, vali_loss))
                torch.save(model_transfer.state_dict(), './drive/My Drive/data/model/modle_best.pt')
                vali_loss_min = vali_loss
            else:
                torch.save(model_transfer.state_dict(), './drive/My Drive/data/model/modle_second.pt')
                vali_loss_second = vali_loss

In [ ]:
model_transfer2 = models.resnext101_32x8d(pretrained=False)

fc = nn.Sequential(OrderedDict([
                          ('(0):Dropout' , nn.Dropout()),
                          ('(1):Linear', nn.Linear(2048, 1000)),
                          ('(2):Batchnorm', nn.BatchNorm1d(1000)), 
                          ('(3):relu', nn.ReLU()),
                          ('(4):Dropout' , nn.Dropout()),
                          ('(5):Linear', nn.Linear(1000, 100)),
                          ('(6):Batchnorm', nn.BatchNorm1d(100)),
                          ('(7):relu', nn.ReLU()),
                          ('(8):Linear', nn.Linear(100,10)),
    
                          ('(9):Batchnorm', nn.BatchNorm1d(10)),
                          ('(10):Linear', nn.Linear(10,5)),
                          ('(11):relu', nn.ReLU()),
    
                          ('(12):output', nn.LogSoftmax(dim=1))
                          ]))

model_transfer2.fc = fc

In [ ]:
model_transfer.load_state_dict(torch.load('./drive/My Drive/data/model/modle_best.pt'))
model_transfer2.load_state_dict(torch.load('./drive/My Drive/data/model/modle_second.pt'))

In [ ]:
test_transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize(MEANS,DEVIATIONS)
                               ])

test_dataset = datasets.ImageFolder('./drive/My Drive/data/test', transform=test_transform)

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32)

In [ ]:
from tqdm import tqdm

In [ ]:
model_transfer.eval()
model_transfer2.eval()
test_loss =0
test_accuracy = 0
with torch.no_grad():
    for x,y in tqdm(test_dataloader):
        
        output = model_transfer(x)
        output2 = model_transfer2(x)
        
        ps = torch.exp(output)
        ps2 = torch.exp(output2)
        ps = (ps+ps2)*0.5
        
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == y.view(*top_class.shape)
        
        test_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

test_accuracy=(test_accuracy/len(test_dataloader))*100

print('정확도 : {:.4f}%'.format(test_accuracy))

100%|██████████| 44/44 [38:17<00:00, 52.22s/it]

정확도 : 0.1420%


In [ ]:
from PIL import Image

import matplotlib.pyplot as plt


def predict(imgpath):
    
    MEANS = [0.5 , 0.5, 0.5]
    DEVIATIONS = [0.5, 0.5 , 0.5]
    transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize(MEANS,DEVIATIONS)
                               ])

    img = Image.open(imgpath).convert('RGB')
    plt.imshow(img)
    img = transform(img)
    img = torch.unsqueeze(img , dim=0)
    
    with torch.no_grad():
        output = model_transfer(img)
        output2 = model_transfer2(img)
    ps = torch.exp(output)
    ps2 = torch.exp(output2)
    ps =(ps+ps2)*0.5
    print(ps)

    top_p, top_class = ps.topk(1, dim=1)
    print(top_class)
    ans = ['공학관', '일송아트홀','산학협력관','대학본부별관', 'clc',]
    print(ans[top_class])

In [ ]:
predict('./drive/My Drive/data/test/clc/img_11.png')